In [1]:
!pip install -q -U trl transformers accelerate peft datasets bitsandbytes evaluate git+https://github.com/huggingface/huggingface_hub


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
from huggingface_hub import notebook_login

notebook_login()

In [60]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/sloberta"

In [61]:
from datasets import load_dataset

dataset = load_dataset("anzeo/Slovene_SuperGLUE_CB")

In [62]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [63]:
from transformers import set_seed

set_seed(42)

id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"

def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [65]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

In [66]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [67]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [68]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [69]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

NUM_VIRTUAL_TOKENS = 12

peft_config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    num_layers=6,
    token_dim=768,
    num_attention_heads=12,
    tokenizer_name_or_path=model_name #The pre-trained model
)

In [70]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 602,115 || all params: 111,226,374 || trainable%: 0.5413


In [71]:
from transformers import TrainingArguments

new_model_name = "prompt_fine_tuned_CB_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-3,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [72]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [73]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=400, training_loss=0.5656847095489502, metrics={'train_runtime': 37.2165, 'train_samples_per_second': 85.983, 'train_steps_per_second': 10.748, 'total_flos': 185378814604152.0, 'train_loss': 0.5656847095489502, 'epoch': 28.571428571428573})

In [74]:
trainer.evaluate()

{'eval_loss': 1.7179303169250488,
 'eval_accuracy': 0.3181818181818182,
 'eval_f1': 0.1590909090909091,
 'eval_runtime': 0.1775,
 'eval_samples_per_second': 123.945,
 'eval_steps_per_second': 16.902,
 'epoch': 28.571428571428573}

In [75]:
trainer.push_to_hub(new_model_name)

C:\Users\anzeo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

events.out.tfevents.1716377489.DESKTOP-22QTFDR.10004.8:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1716377526.DESKTOP-22QTFDR.10004.9:   0%|          | 0.00/457 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anzeo/prompt_fine_tuned_CB_sloberta/commit/b662f479c68adc1762e3eaf4bc0acc55517a3176', commit_message='prompt_fine_tuned_CB_sloberta', commit_description='', oid='b662f479c68adc1762e3eaf4bc0acc55517a3176', pr_url=None, pr_revision=None, pr_num=None)

In [76]:
# Example
hypothesis ="Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [77]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification
import torch

# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)

adapter_name = "anzeo/" + new_model_name

config = PeftConfig.from_pretrained(adapter_name)
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, adapter_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_name)

adapter_config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

In [78]:
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

outputs = model(**inputs, labels=label)
logits = outputs.logits

In [79]:
# Print prediction
logits.argmax().item()

0

In [22]:
# GPU RAM = 1.4 GB